In [49]:
%env THEANO_FLAGS="device=gpu2"
import theano
import lasagne

import theano.tensor as T
import sys
import json
import numpy as np
sys.setrecursionlimit(100000)
floatX = theano.config.floatX
from nltk.stem.snowball import SnowballStemmer
import re
from collections import Counter, defaultdict
from gensim import models
import random
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook, tnrange

env: THEANO_FLAGS="device=gpu2"


## DSSM and question-answering
Here we try to train DSSM model to find the best answer 

In [3]:
#!pip install gensim --upgrade --target=/anaconda3/lib/python3.5/site-packages

In [4]:
stemmer = SnowballStemmer("russian")

In [5]:
questions_all = json.load(open('questions.json'))

In [6]:
questions_train, questions_test = train_test_split(questions_all, test_size = 0.1)

In [7]:
def text_to_tokens(txt):
    txt = txt.lower()
    txt = re.subn('<[a-z\s]*>', '', txt)[0]
    #txt = re.sub('http\w*\s', '', txt)
    r = re.split('(\s|\-|\.|\,)', txt)
    r = (re.subn('[^А-Яа-яA-Za-z\-\.\,\s]', '', _)[0] for _ in r)
    r = [_ for _ in r if len(_)>1 and 'http' not in _]
    return r

In [8]:
corpus = []
tokens = Counter()
for q in tqdm_notebook(questions_train):
    corpus.append(text_to_tokens(q['question']))
    for a in q['answers']:   
        r = text_to_tokens(a['text'])
        corpus.append(r)
        #tokens |= Counter(r)
#len(tokens)

In [9]:
corpus[11]

['почему',
 'диего',
 'симеоне',
 'почти',
 'всегда',
 'на',
 'матче',
 'черном',
 'наряде']

In [10]:
word2vec_size=512

In [11]:
w2v = models.Word2Vec(corpus, min_count=1, size=word2vec_size, workers=10)

In [12]:
1

1

In [13]:
unseen_cnt = 0

In [14]:
def text_to_matrix(seq):
    global unseen_cnt
    matrix = []
    for i, w in enumerate(text_to_tokens(seq)):
        if w not in w2v.wv.vocab:
            matrix.append(np.zeros(word2vec_size))
            unseen_cnt+=1
        else:
            matrix.append(w2v[w])
    return np.matrix(matrix)
text_to_matrix(questions_train[0]['question']).shape

(17, 512)

In [15]:
w2v.most_similar(['король', 'женщина'], ['мужчина'])

[('королева', 0.8363220691680908),
 ('мария', 0.8217662572860718),
 ('анна', 0.7755671739578247),
 ('елизавета', 0.7724213600158691),
 ('екатерина', 0.7592296600341797),
 ('уильям', 0.7533628344535828),
 ('короля', 0.7475455403327942),
 ('принцесса', 0.7453835010528564),
 ('ким', 0.7381781339645386),
 ('бенедикт', 0.7361153364181519)]

In [16]:
epochs = 0

In [17]:
topics = set()
topics_to_id = defaultdict(lambda:0)
id_to_topics = {}
for q in questions_train:
    topics|=set([_.lower() for _ in q['topics']])
for i,t in enumerate(topics):
    topics_to_id[t] = i
    id_to_topics[i] = t

In [18]:
unseen_cnt

0

In [19]:
print(len(topics))
len(questions_train)

8578


111830

In [20]:
#del corpus
import gc
gc.collect()

208

In [21]:
questions_train[1]

{'answers': [],
 'question': 'Что посмотреть из новинок фестивального кино ?',
 'topics': ['Кино']}

In [22]:
def topics_to_vec(cur_topics):
    out = np.zeros((len(topics), ))
    for t in cur_topics:
        out[topics_to_id[t.lower()]] = 1
    return out
topics_to_vec(questions_train[1]['topics']).argmax()

5984

In [23]:
#print(max((_ for _ in enumerate(question_postprocessed)), key=lambda _: len(_[1][1])))

In [24]:
input_questions = T.tensor3('input sequence','float32')
input_positive_answers = T.tensor3('input sequence',"float32")
input_negative_answers = T.tensor3('input sequence','float32')

In [25]:
output_topics = T.matrix('batch topics', 'float32')

In [26]:
target_space_n = 256

In [27]:
class question:
    l = lasagne.layers.InputLayer(shape=(None, None, word2vec_size))
    l = lasagne.layers.LSTMLayer(l, 256, grad_clipping=5, only_return_final=True)
    out = lasagne.layers.DenseLayer(l, target_space_n, nonlinearity=None)

In [28]:
class answer:
    l  = lasagne.layers.InputLayer(shape=(None, None, word2vec_size))
    l = lasagne.layers.LSTMLayer(l, 256, grad_clipping=5, only_return_final=True)
    out = lasagne.layers.DenseLayer(l, target_space_n, nonlinearity=None)

In [29]:
w2v.wv.vocab

{'трансгендер': <gensim.models.keyedvectors.Vocab at 0x7f7122a2a518>,
 'пледы': <gensim.models.keyedvectors.Vocab at 0x7f71224a0fd0>,
 'горембедойнегативом': <gensim.models.keyedvectors.Vocab at 0x7f712cbdb160>,
 'доминорования': <gensim.models.keyedvectors.Vocab at 0x7f7123857ba8>,
 'htmlпара': <gensim.models.keyedvectors.Vocab at 0x7f712d176780>,
 'окислить': <gensim.models.keyedvectors.Vocab at 0x7f712d1766d8>,
 'макэвои': <gensim.models.keyedvectors.Vocab at 0x7f712ce53a90>,
 'sguschenka': <gensim.models.keyedvectors.Vocab at 0x7f712cfef320>,
 'прижало': <gensim.models.keyedvectors.Vocab at 0x7f712cbc5080>,
 'индифференции': <gensim.models.keyedvectors.Vocab at 0x7f712cae8f98>,
 'блефует': <gensim.models.keyedvectors.Vocab at 0x7f712cae8ef0>,
 'коммуникативные': <gensim.models.keyedvectors.Vocab at 0x7f712567beb8>,
 'отказала': <gensim.models.keyedvectors.Vocab at 0x7f712772d400>,
 'сотрясающим': <gensim.models.keyedvectors.Vocab at 0x7f712c51d2b0>,
 'соловьиные': <gensim.models.ke

In [30]:
questions_vec = lasagne.layers.get_output(question.out, inputs=input_questions)
positive_answers_vec = lasagne.layers.get_output(answer.out, inputs=input_positive_answers)
negative_answers_vec = lasagne.layers.get_output(answer.out, inputs=input_negative_answers)



In [31]:
#questions_topic_out.eval({input_questions:X[0:1]}).argmax()

In [32]:
def do_padding(arr):
    '''Takes list, returns padded np array
    (batch_size, longest_seq, w2v_size)
    '''
    size = 0
    for i in arr:
        size = max(size, i.shape[0])
    ret = np.zeros((len(arr), size, word2vec_size), dtype='float32')
    for i,e in enumerate(arr):
        if e.shape[1]!=word2vec_size:
             ret[i] = np.zeros((size,word2vec_size))
        else:
            ret[i] = np.concatenate((np.zeros((size-e.shape[0],word2vec_size)), e), axis=0)
    return ret

In [33]:
def questions_sample_triplet(questions, num):
    '''Returns np sampled array'''
    questions_arr = []
    answers_positive_arr = []
    
    answers_negative_arr = []
    
   
    while len(questions_arr) <num:
        q = random.choice(questions)
        question_vectorized = text_to_matrix(q['question'])
    
        if question_vectorized.shape[0]>0:
            #Positive sampling
            for j, a in enumerate(q['answers']):
                answer_positive_vectorized = text_to_matrix(a['text'])
                #Negative
                q2 = None
                while q2==None or q2==q or len(q2['answers'])==0:
                    q2 = random.choice(questions)
                a = random.choice(q2['answers'])
                answer_negative_vectorized = text_to_matrix(a['text'])
                
                if answer_positive_vectorized.shape[0]>0 and answer_negative_vectorized.shape[0]>0 and question_vectorized.shape[0]>0: 
                    answers_positive_arr.append(answer_positive_vectorized)
                    answers_negative_arr.append(answer_negative_vectorized)
                    questions_arr.append(question_vectorized)
                    
    
    return do_padding(questions_arr[:num]), do_padding(answers_positive_arr[:num]), do_padding(answers_negative_arr[:num])
 
questions_sample_triplet(questions_train, 100)[1].shape

(100, 689, 512)

In [34]:
alpha = 3
print(questions_vec)
loss = T.nnet.relu(((questions_vec - positive_answers_vec)**2).sum(axis=1) - 
              ((questions_vec - negative_answers_vec)**2).sum(axis=1) + alpha).mean()
#loss = (questions_vec + negative_answers_vec + positive_answers_vec).mean()

Elemwise{add,no_inplace}.0


In [35]:
weights = lasagne.layers.get_all_params(answer.out, trainable=True) + lasagne.layers.get_all_params(question.out, trainable=True)
updates = lasagne.updates.adadelta(loss, weights)

In [36]:

#обучение
train = theano.function([input_questions, input_positive_answers, input_negative_answers], loss, updates=updates)

#функция потерь без обучения
compute_cost = theano.function([input_questions, input_positive_answers, input_negative_answers], loss)


In [37]:
transform_answer = theano.function([input_positive_answers], positive_answers_vec)
transform_question = theano.function([input_questions], questions_vec)

In [38]:
batch_size = 50
epoch_size = 10
num_epochs = 500

In [39]:
def get_batch_inputs(batch):
    '''What?'''
    max_len_pos = 0
    current_questions = []
    positive = []
    negative = []
    questions_size = 0
    positive_size = 0
    negative_size = 0
    for question in batch:
        e = answers_postprocessed[random.choice(question[1])]
        current_questions.append(question[0])
        questions_size = max(questions_size, question[0].shape[0])
        positive.append(e)
        positive_size = max(positive_size, e.shape[0])
        e = random.randint(0, len(answers_postprocessed)-1)
        while e in question[1]:
            e = random.randint(0, len(answers_postprocessed))
        e  = answers_postprocessed[e]
        negative.append(e)
        negative_size = max(negative_size, e.shape[0])


    current_questions = a(current_questions, questions_size)
    positive = prepare_batch(positive, positive_size)
    negative = prepare_batch(negative, negative_size)
    return current_questions, positive, negative

In [92]:
def update_moving(moving_average, questions):
    current_questions, positive, negative = questions_sample_triplet(questions, batch_size)
    cost = compute_cost(current_questions, positive, negative)
    print(((transform_question(current_questions)-transform_answer(positive))**2).sum(axis=1).mean())
    if moving_average==-1:
        return cost
    return alpha*cost + (1-alpha)*moving_average

In [94]:
moving_cost_train = -1
moving_cost_test = -1
alpha = 0.1
for i in range(num_epochs):
    for j in range(50):
        current_questions, positive, negative = questions_sample_triplet(questions_train, batch_size)
        train(current_questions, positive, negative)
    moving_cost_train = update_moving(moving_cost_train, questions_train)
    moving_cost_test = update_moving(moving_cost_test, questions_test)
    print("Epoch %s. Train:" % i, moving_cost_train)
    print("Epoch %s. Test:" % i, moving_cost_test)

KeyboardInterrupt: 

In [46]:
11

11

In [ ]:
current_questions, positive, negative = questions_sample_triplet(questions_train, 1)
print(train(current_questions, positive, negative))
print(compute_cost(current_questions, positive, negative))

In [ ]:
compute_cost(current_questions, positive, negative)

# Topics training

In [ ]:
answer_topics = np.zeros((len(answer_ids), len(topics_to_id)))
for i,a in enumerate(tqdm(answer_ids)):
    for t in questions[a[0]]['topics']:
        answer_topics[i, topics_to_id[t.lower()]] = 1
question_topics = np.zeros((len(question_ids), len(topics_to_id)))
for i,a in enumerate(tqdm(question_ids)):
    for t in questions[a]['topics']:
        question_topics[i,topics_to_id[t.lower()]] = 1

In [ ]:
questions_topic_vec = lasagne.layers.get_output(question_topics_model.out, inputs=input_questions)
loss_topics = lasagne.objectives.binary_crossentropy(questions_topic_vec, output_topics).mean()
updates_topics = lasagne.updates.adam(loss_topics, question_topics_model.weights)


train_topics = theano.function([input_questions, output_topics], loss_topics, updates=updates_topics)

#функция потерь без обучения

compute_cost_topics = theano.function([input_questions, output_topics], loss_topics)
#Resulting topic
predict_topics = theano.function([input_questions], questions_topic_vec)

In [ ]:
class question_topics_model:
    weights = []
    l = lasagne.layers.DenseLayer(question.out, 300)
    weights += [l.W, l.b]
    out = lasagne.layers.DenseLayer(l, len(topics), nonlinearity=lasagne.nonlinearities.sigmoid)
    weights += [out.W, out.b]

In [ ]:

def questions_sample(questions, num, positive = True, test=False):
    '''Returns np sampled array'''
    questions_arr = []
    answers_arr = []
    #Positive sampling
    if positive:
        while len(questions_arr) <num:
            q = random.choice(questions)
            question_vectorized = text_to_matrix(q['question'])
            if question_vectorized.shape[0]>0:
                for j, a in enumerate(q['answers']):
                    answer_vectorized = text_to_matrix(a['text'])
                    if answer_vectorized.shape[0]>0: 
                        answers_arr.append(answer_vectorized)
                        questions_arr.append(question_vectorized)
    else:
        while len(questions_arr) <num:
            q = random.choice(questions)
            q2 = None
            while q2==None or q2==q or len(q2['answers'])==0:
                q2 = random.choice(questions)
            a = random.choice(q2['answers'])

            question_vectorized = text_to_matrix(q['question'])
            answer_vectorized = text_to_matrix(a['text'])
            if answer_vectorized.shape[0]>0 and question_vectorized.shape[0]>0: 
                answers_arr.append(answer_vectorized)
                questions_arr.append(question_vectorized)
    questions_arr = questions_arr[:num]
    answers_arr = answers_arr[:num]
    return do_padding(questions_arr), do_padding(answers_arr)


In [ ]:
batch_size_topics = 100
def iterate_minibatches_topics(questions):
    arr = []
    output_topics = []
    for q in questions:
        question_vectorized = text_to_matrix(q['question'])
        if question_vectorized.shape[0]>0:
            cur_topics = np.zeros((1, len(topics_to_id)))
            for t in q['topics']:
                cur_topics[0, topics_to_id[t.lower()]] = 1
            output_topics.append(cur_topics)
            arr.append(question_vectorized)
        while len(arr)>=batch_size_topics:
            yield do_padding(arr[:batch_size_topics]), np.concatenate(output_topics[:batch_size_topics])
            arr = arr[batch_size_topics:]
            output_topics = output_topics[batch_size_topics:]
for i in range(50):       
    for X, Y in iterate_minibatches_topics(questions_train):
        print(train_topics(X, Y.astype('float32')))
    print("Train score at %s" % i, compute_cost_topics(X, Y.astype('float32')))
    for X, Y in iterate_minibatches_topics(questions_test):
        print("Test score at %s" % i, compute_cost_topics(X, Y.astype('float32')))
        break


## Model save

In [ ]:
weight = lasagne.layers.get_all_param_values(question.out)
np.save('weights/question.npy', weight)
weight = lasagne.layers.get_all_param_values(answer.out)
np.save('weights/answer.npy', weight)

In [ ]:
!mkdir weights

In [ ]:
w2v.save('weights/w2v')

## Demo run

In [47]:
len(answers_vectors)

NameError: name 'answers_vectors' is not defined

In [56]:
def get_batch(st):
    return np.array([text_to_matrix(st)])

In [63]:
questions_train[0]

{'answers': [{'name': 'Dmitry Ogar 1776',
   'occupation': 'Аудитор, Москва',
   'text': 'Слушай, ну тут все настолько все индивидуально, что ответить правильно невозможно, все слишком сильно завязано на образ жизни конкретного человека.Мне вот примерно такой суммы хватает на двоих с девушкой (квартира своя, кредитов нет), просто потому что я уже привык жить на эти деньги и все они у меня, можно сказать, расписаны. Но есть знакомый, который к примеру живет в съемной квартире с ребенком и зарплатой 60 тысяч и ему вполне хватает, есть еще один с двумя детьми и примерно такой же зарплатой.Если остаются свободные деньги, которые вы, к примеру, докладываете в банк на вклад или просто храните в копилке, или есть кредит который скоро закроется и появятся свободные средства - можно и детей начать заводить.'}],
 'question': 'Сколько детей не накладно заводить, если общий доход семьи 135к в месяц? При учёте, что квартира съемная и проживание в г. Москва?',
 'topics': ['экономия']}

In [70]:
answers_texts = []
answers_vec = []
for q in tqdm_notebook(questions_train):
    for a in q['answers']:
        m = text_to_matrix(a['text'])
        if m.any():
            answers_texts.append(a)
            answers_vec.append(transform_answer([m]))

In [71]:
transform_question(get_batch('Привет'))

array([[ -4.07315552e-01,   4.04865183e-02,   6.47976696e-02,
          6.34885281e-02,   1.89915046e-01,   1.78984746e-01,
          5.44094920e-01,  -2.69769400e-01,  -1.27000496e-01,
          5.22656925e-02,  -3.90352681e-02,   8.77708793e-02,
         -2.86024243e-01,  -3.74233693e-01,   1.70735627e-01,
          1.44963115e-01,  -1.00583181e-01,   2.18076840e-01,
          5.46675026e-01,  -7.81701319e-03,   3.57381940e-01,
          1.65677965e-01,   1.83024585e-01,  -2.21205205e-01,
         -1.78733706e-01,  -1.15068316e-01,   1.02820456e-01,
         -3.10941190e-01,  -1.83340505e-01,   6.23580776e-02,
         -4.34500545e-01,   2.99891710e-01,   1.67353880e-02,
         -8.23773667e-02,   1.48067638e-01,  -1.18740477e-01,
         -5.98411262e-02,  -8.43895450e-02,  -1.72729298e-01,
         -9.58193615e-02,   5.14079183e-02,   3.81327942e-02,
          1.02199763e-01,  -5.05728163e-02,  -5.22887632e-02,
         -1.47717237e-01,   3.43595505e-01,  -4.89675179e-02,
        

In [81]:
def answer(st=None):
    if not st:
        st = np.random.choice(questions_train)['question'] 
    question_batch = np.array([text_to_matrix(st)])
    question_vector = transform_question(question_batch)[0]
    min_id = 0
    min_dist = 10**9
    
    #t = predict_topic(a)[0]
    for i in tnrange(len(answers_texts)):
        txt, answer_vector = answers_texts[i], answers_vec[i] 
        d = ((question_vector-answer_vector)**2).sum()
        if d<min_dist:
            min_dist = d
            min_id = i
    print(st)
    print(min_dist)
    print(answers_texts[min_id])

#    print(questions[min_id[0]][2][min_id[1]-1])
#answer('', rand=True)

In [95]:
answer('')


Есть ли альтернатива В.В.Путину?!
8.36932
{'occupation': 'Лазаю по крышам, цепляюсь за поезда, гуляю с #ФСБ и просто пишу посты для vk.com/brown_room', 'text': 'Смотря где. В России, как знаете, Государственная дума решила, что грешно, приняв закон с кучей поправок "в целях противодействия оскорблению религиозных убеждений и чувств граждан". Годик можно посидеть или 10-30 тысяч заплатить штрафа.А вот в Нидерландах года 3 назад отменили подобный закон.', 'name': 'Andrej Biryukov 799'}


In [96]:
answer('')


В банкоматах какого банка в России выгоднее всего снимать доллары?
13.908
{'occupation': 'Специалист широкого профиля', 'text': 'Из тех, вещей, что действительно можно купить – это компания Apple, чтобы приобрести её (то есть 100% её акций) вам потребуется более триллиона долларов США. Сейчас капитализация компании чуть меньше, но при полной покупке вам придётся заплатить существующим акционерам больше рыночной цены за каждую акцию.', 'name': 'Artem Gordin 1667'}


In [98]:
answer('В чем смысл математики?')


В чем смысл математики?
23.3876
{'occupation': 'Студент кафедры физики частиц и космологии МГУ', 'text': 'Присоединяюсь к коллеге. Вообще очень сложно объяснить неспециалисту зачем нужен бозон Хиггса, почему именно он должен сообщать массу частицам. Суть дела лежит с структуре самой стандартной модели и в ее теоретической базе - теории калибровочных полей. На мой взгляд, очень удачная попытка объяснения была сделана известным теоретиком Хуаном Малдасеной в статье arxiv.org Статья простая и понятная, содержит около 20 страниц текста, но, к сожалению, она на английском. Так что если вам интересно более-менее в деталях узнать на чем основана физика частиц, у вас есть свободное время и вы немного владеете английским, то очень рекомендую статью почитать.', 'name': 'Николай Сухов 271'}


In [99]:
answer('Куда съездить в путешествие?')


Куда съездить в путешествие?
20.5129
{'occupation': 'программист, предприниматель', 'text': 'Суздаль, Екатеринбург, Калининград, Казань, Набережные Челны.\xa0Города красивые, есть что посмотреть, есть чего попробовать. Рекомендую в общем.\xa0', 'name': 'Илья Левин 4832'}
